<a href="https://colab.research.google.com/github/hrensimin05/DATA608/blob/main/Module4_608.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:

### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Install Dash
!pip install dash==0.31.1  # The core dash backend
!pip install dash-html-components==0.13.2  # HTML components
!pip install dash-core-components==0.39.0  # Supercharged components
!pip install dash-table==3.1.7  # Interactive DataTable component (new!)

--2021-10-25 04:00:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.19M  5.60MB/s    in 2.4s    

2021-10-25 04:00:34 (5.60 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
http://61bb-35-221-223-237.ngrok.io


In [39]:
%%writefile my_app1.py

import pandas as pd
import numpy as np
import json
import plotly.offline as py
import plotly.graph_objs as go
from plotly import subplots


import dash
import dash_core_components as dcc
import dash_html_components as html

url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'


data = pd.read_json(url)
#Question One

q1 = data[['spc_common','health','boroname']]
q1['spc_common'].fillna('Unknown',inplace = True)
q1.dropna(inplace = True)

#finding health conditions
statuses = list(set(q1['health']))
colors = ['rgb(0, 204, 102)','rgb(51, 102, 0)','rgb(0, 255, 0)']

#create columns 
for status in set(q1['health']):
    q1[status] = np.where(q1['health']==status,1,0)
    
q1 = pd.DataFrame(q1.groupby(['boroname','spc_common']).sum())


boroughs = list(set(data['boroname']))

#calculate proportion of trees in different conditions
q1['total'] = q1.sum(axis=1)
for column in list(q1.columns):
    q1[column] = (q1[column]/q1['total'])*100
q1.head()

#create list 
q1_list = []

#create plot titles
br_list = list(map(lambda x: str(x), boroughs))
row = 1
col = len(boroughs)

f1 = subplots.make_subplots(rows=row, cols=col, subplot_titles=tuple(br_list))


for borough in boroughs:
        for i in range(0,len(statuses)):
            trace = go.Bar(
            x = list(q1.loc[borough].index),
            y = list(q1.loc[borough][statuses[i]]),
            name = statuses[i],
            marker=dict(color=colors[i])
            )
            q1_list += [trace]

row_i = []
col_j = []
for i in range(1,row+1):
    for j in range (1,col+1):
        for n in range (1,4):
            row_i.append(i)
            col_j.append(j)

for i in range(0,len(q1_list)):        
     f1.append_trace(q1_list[i], row_i[i],col_j[i]) 
 
        
f1['layout'].update(showlegend=False,height=400, width=1400, title='Segments of Trees in Good, Fair and Poor Conditions', barmode='stack')





#Question 2
q2 = data[['spc_common','health','boroname','steward']]
q2['spc_common'].fillna('Unknown',inplace = True)
q2.dropna(inplace = True)
q2[['steward','health']] = q2[['steward','health']].apply(lambda x : pd.factorize(x)[0])
q2_df = pd.DataFrame(q2.groupby(['boroname','spc_common']).corr())


f2 = subplots.make_subplots(rows=1, cols=len(boroughs), subplot_titles=tuple(br_list))

#create list that stores graphs data
q2_list =[]


data = list(set(q2['spc_common']))


for borough in boroughs:
    trace = go.Bar(  
            x = list(q1.loc[borough].index),
            y = list(q2_df.loc[borough]['steward'][::2])
            )
    q2_list += [trace]


for i in range(len(boroughs)):
    f2.append_trace(q2_list[i], 1, i+1) 


f2['layout'].update(showlegend=False,height=500, width=1400, title='Link between stewards and health of trees')


app = dash.Dash()

colors = {
    'background': '#ffffff',
    'text': '#282828'
}



app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Question #1',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    html.Div(children='Segments of trees in Good, Fair and Poor conditions', style={
        'textAlign': 'center',
        'color': colors['text']
    }),
    
    html.Div([
        dcc.Graph(figure=f1, id='my-figure-q1')
    ]),

    html.H1(
        children='Question #2',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    
    html.Div(children='Link between stewards and health of trees', style={
        'textAlign': 'center',
        'color': colors['text']
    }),
    
    html.Div([
        dcc.Graph(figure=f2, id='my-figure-q2')
    ])

    ])



if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting my_app1.py


In [ ]:
!python my_app1.py

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

my_app1.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

my_app1.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from 